In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import re

In [3]:
from zipfile import ZipFile
with ZipFile('20k_dataset.zip', 'r') as zf:
    zf.extractall('./')

In [51]:
news_train_df = pd.read_csv('./train.csv')
news_train_df.head(20)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [94]:
news_test_df = pd.read_csv('./test.csv')
news_test_df

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [77]:
bool_series_t = pd.notnull(news_train_df['title'])
news_train_df[bool_series_t]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...,...
20198,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20199,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20200,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20201,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [79]:
news_train_df = news_train_df[bool_series_t].reset_index(drop =True)
news_train_df= news_train_df.reset_index(drop=True)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...,...
20198,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20199,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20200,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20201,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [80]:
news_train_df.drop(columns= ['id'])
news_train_df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...,...
20198,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20199,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20200,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20201,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [81]:
bool_series_text = pd.notnull(news_train_df['text'])
news_train_df = news_train_df[bool_series_text]

In [82]:
news_train_df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...,...
20198,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20199,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20200,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20201,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [83]:
bool_series_text = pd.notnull(news_train_df['text'])
news_train_df = news_train_df[bool_series_text]
news_train_df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...,...
20198,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20199,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20200,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20201,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [84]:
news_train_df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [89]:
news_train_df=news_train_df.drop(columns = ['id'])

In [90]:
news_train_df

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...
20198,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20199,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20200,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20201,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [86]:
from wordcloud import STOPWORDS
STOPWORDS.add("rt")
STOPWORDS.add("s")
STOPWORDS.add("u")
STOPWORDS.add("amp")
STOPWORDS.add("th")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("m")

In [87]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiword

True

In [91]:
import re
for i in range(len(news_train_df)):
    d = news_train_df.loc[i]["text"]
    d=re.sub(r'http\S+', '', d) #remove links
    d=re.sub("[^a-zA-Z]", " ", d) #remove all characters except letters
    d=d.lower() #convert all words to lowercase
    d=nltk.word_tokenize(d) #split sentences into word
    d=[word for word in d if not word in STOPWORDS] #remove the stopwords
    lemma=nltk.WordNetLemmatizer() 
    d=[lemma.lemmatize(word) for word in d] #identify the correct form of the word in the dictionary
    d=" ".join(d)
    
    news_train_df.at[i,"text"]=d

In [106]:
news_train_df

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide didn even see comey letter jaso...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,feeling life circle roundabout rather head str...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might fired october tension intelligence...,1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,video civilian killed single airstrike identif...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six year prison ...,1
...,...,...,...,...
20198,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,rapper unloaded black celebrity met donald tru...,0
20199,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,green bay packer lost washington redskin week ...,0
20200,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,macy today grew union several great name ameri...,0
20201,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,nato russia hold parallel exercise balkan pres...,1


In [111]:
news_training_df  = news_train_df.iloc[:15000,:]
news_testing_df = news_train_df.iloc[15001:,:]
news_testing_df

,title,author,text,label
15001,Monsieur Malbrough est Mort | New Eastern Outlook,Author,region europe recently published article title...,1
15002,C.I.A. Chief Warns Donald Trump Against Tearin...,Dan Bilefsky,london director c issued stark warning donald ...,0
15003,Seattle Doling Legal Aid for Illegal Aliens,John Binder,taxpayer seattle soon paying illegal alien rec...,0
15004,This Honest Trailer for Sherlock is hilarious ...,OK,next prev swipe left right honest trailer sher...,1
15005,Hillary Clinton Jumps the Shark with ‘Trump’s ...,NaN,st century wire say yesterday wikileaks editor...,1
...,...,...,...,...
20198,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,rapper unloaded black celebrity met donald tru...,0
20199,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,green bay packer lost washington redskin week ...,0
20200,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,macy today grew union several great name ameri...,0
20201,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,nato russia hold parallel exercise balkan pres...,1


In [100]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

def token(text):
    txt = nltk.word_tokenize(text.lower())
    return [st.stem(word) for word in txt]

In [101]:
#cv = CountVectorizer(lowercase=True, 
#                     tokenizer=token, stop_words=STOPWORDS,
#                     analyzer=u'word', min_df=4)

tv = TfidfVectorizer(stop_words = STOPWORDS,
                     lowercase=True, binary = True)

In [112]:
vec_train = tv.fit_transform(news_training_df['text'])
vec_test = tv.transform(news_testing_df['text'])

In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report


rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rfc.fit(X = vec_train, y = news_training_df['label'])

y_pred_rfc=rfc.predict(vec_test)
#accuracy_score(y_pred_rfc, test_target)


In [114]:
rfc_report=classification_report(news_testing_df['label'],y_pred_rfc)
print(rfc_report)

              precision    recall  f1-score   support

           0       0.87      0.94      0.91      2657
           1       0.93      0.86      0.89      2545

    accuracy                           0.90      5202
   macro avg       0.90      0.90      0.90      5202
weighted avg       0.90      0.90      0.90      5202



In [118]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=25000, n_features=4, n_informative=3, n_redundant=0, random_state=1)
model = RandomForestClassifier()
model.fit(X, y)
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
print(X ,y)

Feature: 0, Score: 0.12597
Feature: 1, Score: 0.02453
Feature: 2, Score: 0.72096
Feature: 3, Score: 0.12853
[[-0.08475204 -2.11716863  0.85175395 -1.03890271]
 [-2.71598525 -1.09189784 -1.95020879  0.46621376]
 [ 0.97293604 -0.34125398 -0.0778616   2.36914395]
 ...
 [ 2.31358491  0.39831978  1.76317605  1.27989382]
 [ 0.80842384 -0.39330673  0.4746392   1.37447403]
 [-1.65783831 -0.27461375 -2.19101279  3.08144064]] [0 1 1 ... 0 0 1]
